In [2]:
# Aadil Nazir
import optuna
import torch
import torchvision
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def objective(trial):
    # HP search space
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd'])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int('batch_size', 16, 64)

    print(f"Trial {trial.number}: optimizer={optimizer_name}, lr={learning_rate}, batch_size={batch_size}")

    # Data transformation and loading
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    trainset = CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)

    # Model setup
    model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT) # weight updated 
    model.fc = nn.Linear(model.fc.in_features, 10)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) if optimizer_name == 'adam' else \
                optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

    # Training loop
    num_epochs = 2
    for epoch in range(num_epochs):
        print(f"Starting epoch {epoch+1}...")
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(trainloader):
            if i % 10 == 0:  # log every 10 batches
                print(f"Batch {i}/{len(trainloader)}...")
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1} completed. Loss: {running_loss / len(trainloader)}")

    return running_loss / len(trainloader)

#optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5, timeout=300) # increased

print("Best hyperparameters: ", study.best_params)

[I 2024-12-14 11:47:36,716] A new study created in memory with name: no-name-54fb0611-7441-4d3f-bad7-bedbedcc88cb


Trial 0: optimizer=sgd, lr=0.0028299879076974566, batch_size=39
Files already downloaded and verified
Starting epoch 1...
Batch 0/1283...
Batch 10/1283...
Batch 20/1283...
Batch 30/1283...
Batch 40/1283...
Batch 50/1283...
Batch 60/1283...
Batch 70/1283...
Batch 80/1283...
Batch 90/1283...
Batch 100/1283...
Batch 110/1283...
Batch 120/1283...
Batch 130/1283...
Batch 140/1283...
Batch 150/1283...
Batch 160/1283...
Batch 170/1283...
Batch 180/1283...
Batch 190/1283...
Batch 200/1283...
Batch 210/1283...
Batch 220/1283...
Batch 230/1283...
Batch 240/1283...
Batch 250/1283...
Batch 260/1283...
Batch 270/1283...
Batch 280/1283...
Batch 290/1283...
Batch 300/1283...
Batch 310/1283...
Batch 320/1283...
Batch 330/1283...
Batch 340/1283...
Batch 350/1283...
Batch 360/1283...
Batch 370/1283...
Batch 380/1283...
Batch 390/1283...
Batch 400/1283...
Batch 410/1283...
Batch 420/1283...
Batch 430/1283...
Batch 440/1283...
Batch 450/1283...
Batch 460/1283...
Batch 470/1283...
Batch 480/1283...
Batch 4

[I 2024-12-14 12:41:53,635] Trial 0 finished with value: 0.6268795250057709 and parameters: {'optimizer': 'sgd', 'learning_rate': 0.0028299879076974566, 'batch_size': 39}. Best is trial 0 with value: 0.6268795250057709.


Epoch 2 completed. Loss: 0.6268795250057709
Best hyperparameters:  {'optimizer': 'sgd', 'learning_rate': 0.0028299879076974566, 'batch_size': 39}
